In [2]:
'''
Three IDs: userID, creativeID, positionID
'''


import os
import json
import numpy as np
import pandas as pd
from time import time, strftime
from collections import Counter
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
print(' >>>>>>>>> Devv stat 1577799 >>>>>>>>>>>> ')

def save_preds(preds, cb=False):
    preds = np.ravel(preds)
    assert len(preds)==338489
    avg = np.average(preds)
    std = np.std(preds)
    p = '../%s%s_dnn_tl_result_%.4f_%.4f_%s.csv' % ('callback_' if cb else '', 
        strftime('%H%M_%m%d'), avg, std, 'args.m')

    df = pd.DataFrame({'instanceID': te_df_['instanceID'].values, 'proba': preds})
    df.sort_values('instanceID', inplace=True)
    df.to_csv(p, index=False)

    if cb: 
        print(' Written to: ', p)
        return avg, std
    else:
        print('\nTrain average: ', tr_avg)
        print('Preds average: ', avg)
        print('Preds std dev.: ', std)
        print('\nWritten to: ', p)


'''
OrderedDict([('conversionTime_d', 0.25819888974716115),
             ('userID', 0.16155678776166404),
             ('appCategory', 0.11342039643011995),
             ('positionID', 0.097800108293958354),
             ('positionType', 0.087460470534174911),
             ('creativeID', 0.071309827665513831),
             ('appID', 0.06958037043067912),
             ('adID', 0.063169867995414214),
             ('advertiserID', 0.05492409862736055),
             ('camgaignID', 0.050796291439061121),
             ('sitesetID', 0.013700972348893198),
             ('connectionType', 0.010308483261562163),
             ('residence', 0.0075065408905755091),
             ('age', 0.006662250035074235),
             ('hometown', 0.0057415423374340075),
             ('haveBaby', 0.0050835138043705281),
             ('telecomsOperator', 0.0049696062876111837),
             ('gender', 0.0046607920728349225),
             ('education', 0.0030160253068509456),
             ('clickTime_h', 0.0029165451700999038),
             ('clickTime_d', 0.0028077837477102278),
             ('weekDay', 0.001772181032721335),
             ('marriageStatus', 0.0016869540469555094),
             ('appPlatform', 0.0008981783007564663),
             ('clickTime_m', 0.00072661907607473827)])
'''
# ====================================================================================== #
# ====================================================================================== #
# data
# features = ['appCategory', 'positionID', 'positionType', 'creativeID', 'appID', 'adID',
#             'advertiserID', 'camgaignID', 'sitesetID', 'connectionType',
#             'residence', 'age', 'hometown', 'haveBaby', 'telecomsOperator',
#             'gender', 'education', 'clickTime_h', 'clickTime_d', 'weekDay',
#             'marriageStatus', 'appPlatform', 'clickTime_m', 'userID']

cate_features = ['appCategory', 'positionType', 'connectionType', 'haveBaby', 'telecomsOperator',
            'gender', 'education', 'clickTime_h', 'weekDay',
            'marriageStatus', 'appPlatform']

features = ['advertiserID', 'sitesetID', 'age']
# features = ['connectionType', 'telecomsOperator', 'appPlatform', 'gender',
#        'education', 'marriageStatus', 'haveBaby', 'sitesetID', 'positionType',
#        'weekDay']
features.reverse()

tr_df = pd.read_csv('../data/pre/new_generated_train.csv')
te_df_ = pd.read_csv('../data/pre/new_generated_test.csv')
va_df = tr_df.loc[tr_df['clickTime_d'] == 24]

print(tr_df.head())

tr_ui = pd.read_csv('../data/pre/new_tr_ui.csv', header=None).values
te_ui = pd.read_csv('../data/pre/new_te_ui.csv', header=None).values
va_ui = pd.read_csv('../data/pre/new_va_ui.csv', header=None).values

tr_ua = pd.read_csv('../data/pre/new_tr_ua.csv', header=None).values
te_ua = pd.read_csv('../data/pre/new_te_ua.csv', header=None).values
va_ua = pd.read_csv('../data/pre/new_va_ua.csv', header=None).values

 >>>>>>>>> Devv stat 1577799 >>>>>>>>>>>> 
   Unnamed: 0  label  creativeID   userID  positionID  connectionType  \
0           0      0        3089  2798058         293               1   
1           1      0        1259   463234        6161               1   
2           2      0        4465  1857485        7434               4   
3           3      0        1004  2038823         977               1   
4           4      0        1887  2015141        3688               1   

   telecomsOperator  adID  camgaignID  advertiserID        ...         \
0                 1  1321          83            10        ...          
1                 2  1535         685            80        ...          
2                 1   147         460             3        ...          
3                 1   411         564             3        ...          
4                 1   369         144            84        ...          

   hometown  residence  sitesetID  positionType  appCategory  clickTime_d  \
0 

In [21]:
x_all = pd.concat([tr_df[cate_features], te_df_[cate_features], va_df[cate_features]])

In [23]:
encoded_x = None
for f in cate_features:
    feature = x_all[f].values.reshape(-1,1)
    onehot_encoder = OneHotEncoder(sparse=False)
    feature = onehot_encoder.fit_transform(feature)
    if encoded_x is None:
        encoded_x = feature
    else:
        encoded_x = np.concatenate((encoded_x, feature), axis=1)
print("\nX shape: ", encoded_x.shape)


X shape:  (4373259, 84)


In [27]:
tr_x = np.concatenate((encoded_x[:len(tr_df)], tr_ui, tr_ua, tr_df[features].values), axis=1)


In [31]:
encoded_x

array([[ 0.,  0.,  0., ...,  1.,  1.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]])

In [36]:
encoded_x[-len(va_df):].shape

(285242, 84)

In [37]:
e_x = np.concatenate((encoded_x[len(te_df_):len(te_df_)+len(va_df)], te_ui, te_ua, te_df_[features].values), axis=1)


ValueError: all the input array dimensions except for the concatenation axis must match exactly